In [64]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sn

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

import warnings
warnings.filterwarnings('ignore')

# Any results you write to the current directory are saved as output.

sample_submission.csv
test.csv
train.csv



In [111]:
df = pd.read_csv('../input/train.csv', parse_dates=["release_date"])
#df = df.dropna()
df.shape
test = pd.read_csv('../input/test.csv', parse_dates=["release_date"])
#df.head()

In [112]:
def isengorzh(x):
    if 'en' in x or 'zh' in x:
        return True
    else:
        return False

In [113]:
df['year'] = df['release_date'].dt.year
#data['day'] = data['release_date'].dt.day
#df['month'] = df['release_date'].dt.month
#df['release_dayofweek'] = df['release_date'].dt.dayofweek
#df['release_quarter'] = df['release_date'].dt.quarter
df['belongs_to_collection'] = df['belongs_to_collection'].apply(lambda x: isinstance(x, str))
df['tagline'] = df['tagline'].apply(lambda x: isinstance(x, str))
df['homepage'] = df['homepage'].apply(lambda x: isinstance(x, str))
df['runtime'].fillna(df['runtime'].mean(), inplace=True)
df.loc[df['year'] > 2018, 'year'] = df.loc[df['year'] > 2018, 'year'].apply(lambda x: x - 100)
df['_budget_year_ratio'] = df['budget']/(df['year']*df['year'])
df['is_original_language_en_or_zh'] = df['original_language'].apply(lambda x: isengorzh(x))
df.head()

,id,belongs_to_collection,budget,genres,homepage,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,runtime,spoken_languages,status,tagline,title,Keywords,cast,crew,revenue,year,_budget_year_ratio,is_original_language_en_or_zh
0,1,True,14000000,"[{'id': 35, 'name': 'Comedy'}]",False,tt2637294,en,Hot Tub Time Machine 2,"When Lou, who has become the ""father of the In...",6.575393,/tQtWuwvMf0hCc2QR2tkolwl7c3c.jpg,"[{'name': 'Paramount Pictures', 'id': 4}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",2015-02-20,93.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,True,Hot Tub Time Machine 2,"[{'id': 4379, 'name': 'time travel'}, {'id': 9...","[{'cast_id': 4, 'character': 'Lou', 'credit_id...","[{'credit_id': '59ac067c92514107af02c8c8', 'de...",12314651,2015,3.448085,True
1,2,True,40000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",False,tt0368933,en,The Princess Diaries 2: Royal Engagement,Mia Thermopolis is now a college graduate and ...,8.248895,/w9Z7A0GHEhIp7etpj0vyKOeU1Wx.jpg,"[{'name': 'Walt Disney Pictures', 'id': 2}]","[{'iso_3166_1': 'US', 'name': 'United States o...",2004-08-06,113.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,True,The Princess Diaries 2: Royal Engagement,"[{'id': 2505, 'name': 'coronation'}, {'id': 42...","[{'cast_id': 1, 'character': 'Mia Thermopolis'...","[{'credit_id': '52fe43fe9251416c7502563d', 'de...",95149435,2004,9.960120,True
2,3,False,3300000,"[{'id': 18, 'name': 'Drama'}]",True,tt2582802,en,Whiplash,"Under the direction of a ruthless instructor, ...",64.299990,/lIv1QinFqz4dlp5U4lQ6HaiskOZ.jpg,"[{'name': 'Bold Films', 'id': 2266}, {'name': ...","[{'iso_3166_1': 'US', 'name': 'United States o...",2014-10-10,105.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,True,Whiplash,"[{'id': 1416, 'name': 'jazz'}, {'id': 1523, 'n...","[{'cast_id': 5, 'character': 'Andrew Neimann',...","[{'credit_id': '54d5356ec3a3683ba0000039', 'de...",13092000,2014,0.813570,True
3,4,False,1200000,"[{'id': 53, 'name': 'Thriller'}, {'id': 18, 'n...",True,tt1821480,hi,Kahaani,Vidya Bagchi (Vidya Balan) arrives in Kolkata ...,3.174936,/aTXRaPrWSinhcmCrcfJK17urp3F.jpg,NaN,"[{'iso_3166_1': 'IN', 'name': 'India'}]",2012-03-09,122.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,False,Kahaani,"[{'id': 10092, 'name': 'mystery'}, {'id': 1054...","[{'cast_id': 1, 'character': 'Vidya Bagchi', '...","[{'credit_id': '52fe48779251416c9108d6eb', 'de...",16000000,2012,0.296432,False
4,5,False,0,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",False,tt1380152,ko,마린보이,Marine Boy is the story of a former national s...,1.148070,/m22s7zvkVFDU9ir56PiiqIEWFdT.jpg,NaN,"[{'iso_3166_1': 'KR', 'name': 'South Korea'}]",2009-02-05,118.0,"[{'iso_639_1': 'ko', 'name': '한국어/조선말'}]",Released,False,Marine Boy,NaN,"[{'cast_id': 3, 'character': 'Chun-soo', 'cred...","[{'credit_id': '52fe464b9251416c75073b43', 'de...",3923970,2009,0.000000,False


Кажется, все фичи с корреляцией меньше 0.20 текущую модуль ухудшают.

In [114]:
df[['_budget_year_ratio','popularity','runtime','belongs_to_collection','tagline','is_original_language_en_or_zh','revenue']].corr()

,_budget_year_ratio,popularity,runtime,belongs_to_collection,tagline,is_original_language_en_or_zh,revenue
_budget_year_ratio,1.000000,0.341152,0.239241,0.223298,0.218354,0.190304,0.752273
popularity,0.341152,1.000000,0.133669,0.155896,0.164770,0.121088,0.461460
runtime,0.239241,0.133669,1.000000,-0.036916,0.011485,-0.087838,0.216363
belongs_to_collection,0.223298,0.155896,-0.036916,1.000000,0.112840,0.006662,0.339425
tagline,0.218354,0.164770,0.011485,0.112840,1.000000,0.383688,0.180366
is_original_language_en_or_zh,0.190304,0.121088,-0.087838,0.006662,0.383688,1.000000,0.145376
revenue,0.752273,0.461460,0.216363,0.339425,0.180366,0.145376,1.000000


In [115]:
from sklearn.model_selection import train_test_split

y_val = df.revenue
Y= y_val.apply(np.log10)
feat_movie  = ['_budget_year_ratio','popularity','runtime',
                     'belongs_to_collection','tagline',
               #'is_original_language_en_or_zh'
              ]
feat = df[feat_movie]
feat.runtime = pd.get_dummies(feat.runtime)
print(feat.isnull().sum())
X_train, X_test, Y_train, Y_test = train_test_split(feat,Y, test_size=0.3,random_state=4)

_budget_year_ratio               0
popularity                       0
runtime                          0
belongs_to_collection            0
tagline                          0
is_original_language_en_or_zh    0
dtype: int64


In [69]:
params = {'objective':'regression',
          'num_leaves' : 40,
          'min_data_in_leaf' : 10,
          'max_depth' : 6,
          'learning_rate': 0.001,
          "metric": 'rmse',
          "random_state" : 42,
          "lambda_l2" : 0.005,
          "verbosity": -1}

In [116]:
test = pd.read_csv('../input/test.csv', parse_dates=["release_date"])

test['year'] = test['release_date'].dt.year
test['month'] = test['release_date'].dt.month
test['release_dayofweek'] = test['release_date'].dt.dayofweek
test['release_quarter'] = test['release_date'].dt.quarter
test['year'].fillna(test['year'].mean(), inplace=True)
test['month'].fillna(test['month'].mean(), inplace=True)
test['release_dayofweek'].fillna(test['release_dayofweek'].mean(), inplace=True)
test['release_quarter'].fillna(test['release_quarter'].mean(), inplace=True)
test['belongs_to_collection'] = test['belongs_to_collection'].apply(lambda x: isinstance(x, str))
test['tagline'] = test['tagline'].apply(lambda x: isinstance(x, str))
test['homepage'] = test['homepage'].apply(lambda x: isinstance(x, str))
test['runtime'].fillna(test['runtime'].mean(), inplace=True)
test.loc[test['year'] > 2018, 'year'] = test.loc[test['year'] > 2018, 'year'].apply(lambda x: x - 100)
test['_budget_year_ratio'] = test['budget']/(test['year']*test['year'])
#test['is_original_language_en_or_zh'] = test['original_language'].apply(lambda x: isengorzh(x))

X_t = test[feat_movie]
X_t.shape

(4398, 6)

In [71]:
#from sklearn.ensemble import RandomForestRegressor
#from sklearn.model_selection import train_test_split
#from sklearn.metrics import mean_squared_log_error
#from math import sqrt

#X_train, X_test, y_train, y_test = train_test_split(feat, Y, test_size=0.25, random_state=42)

#RFRegressor = RandomForestRegressor(
#    max_features=3, 
#    n_estimators=100, 
#    random_state=42)
#RFRegressor.fit(X_train, y_train)
#RFpredictions = RFRegressor.predict(X_test)
#
#RFscore = mean_squared_log_error(y_test, RFpredictions)
#print(sqrt(score))

#RF_predictions = RFRegressor.predict(X_test)

In [72]:
#from xgboost import XGBRegressor
#xgb_model = XGBRegressor(n_estimators=1000)
#xgb_model.fit(X_train, y_train, early_stopping_rounds=5, eval_set=[(X_test, y_test)], verbose=False)
#xgb_val_predictions = xgb_model.predict(X_test)

#XGBscore = mean_squared_log_error(y_test, xgb_val_predictions)
#print(sqrt(XGBscore))

In [73]:
import lightgbm as lgbm

lgbm_train = lgbm.Dataset(X_train,Y_train)
lgbm_eval = lgbm.Dataset(X_test,Y_test,reference = lgbm_train)
print(X_train.shape)
print(X_t.shape)

(2100, 5)
(4398, 5)


In [74]:
print("TRAINING: ")

gbm = lgbm.train(params,lgbm_train,num_boost_round=3000,valid_sets = lgbm_eval,early_stopping_rounds=15)

TRAINING: 
[1]	valid_0's rmse: 1.29588
Training until validation scores don't improve for 15 rounds.
[2]	valid_0's rmse: 1.29529
[3]	valid_0's rmse: 1.29471
[4]	valid_0's rmse: 1.29413
[5]	valid_0's rmse: 1.29354
[6]	valid_0's rmse: 1.29296
[7]	valid_0's rmse: 1.29238
[8]	valid_0's rmse: 1.29181
[9]	valid_0's rmse: 1.29123
[10]	valid_0's rmse: 1.29065
[11]	valid_0's rmse: 1.29008
[12]	valid_0's rmse: 1.2895
[13]	valid_0's rmse: 1.28893
[14]	valid_0's rmse: 1.28836
[15]	valid_0's rmse: 1.28778
[16]	valid_0's rmse: 1.28721
[17]	valid_0's rmse: 1.28664
[18]	valid_0's rmse: 1.28608
[19]	valid_0's rmse: 1.28551
[20]	valid_0's rmse: 1.28494
[21]	valid_0's rmse: 1.28438
[22]	valid_0's rmse: 1.28381
[23]	valid_0's rmse: 1.28325
[24]	valid_0's rmse: 1.28269
[25]	valid_0's rmse: 1.28212
[26]	valid_0's rmse: 1.28156
[27]	valid_0's rmse: 1.281
[28]	valid_0's rmse: 1.28046
[29]	valid_0's rmse: 1.2799
[30]	valid_0's rmse: 1.27936
[31]	valid_0's rmse: 1.2788
[32]	valid_0's rmse: 1.27826
[33]	valid_0'

[411]	valid_0's rmse: 1.12257
[412]	valid_0's rmse: 1.12226
[413]	valid_0's rmse: 1.12196
[414]	valid_0's rmse: 1.12167
[415]	valid_0's rmse: 1.12137
[416]	valid_0's rmse: 1.12107
[417]	valid_0's rmse: 1.12077
[418]	valid_0's rmse: 1.12047
[419]	valid_0's rmse: 1.12018
[420]	valid_0's rmse: 1.11989
[421]	valid_0's rmse: 1.11959
[422]	valid_0's rmse: 1.1193
[423]	valid_0's rmse: 1.11901
[424]	valid_0's rmse: 1.11871
[425]	valid_0's rmse: 1.11842
[426]	valid_0's rmse: 1.11813
[427]	valid_0's rmse: 1.11783
[428]	valid_0's rmse: 1.11754
[429]	valid_0's rmse: 1.11725
[430]	valid_0's rmse: 1.11696
[431]	valid_0's rmse: 1.11667
[432]	valid_0's rmse: 1.11638
[433]	valid_0's rmse: 1.1161
[434]	valid_0's rmse: 1.11581
[435]	valid_0's rmse: 1.11552
[436]	valid_0's rmse: 1.11524
[437]	valid_0's rmse: 1.11495
[438]	valid_0's rmse: 1.11466
[439]	valid_0's rmse: 1.11438
[440]	valid_0's rmse: 1.11409
[441]	valid_0's rmse: 1.11381
[442]	valid_0's rmse: 1.11352
[443]	valid_0's rmse: 1.11324
[444]	valid_

[705]	valid_0's rmse: 1.05164
[706]	valid_0's rmse: 1.05144
[707]	valid_0's rmse: 1.05123
[708]	valid_0's rmse: 1.05105
[709]	valid_0's rmse: 1.05087
[710]	valid_0's rmse: 1.05068
[711]	valid_0's rmse: 1.05047
[712]	valid_0's rmse: 1.05027
[713]	valid_0's rmse: 1.05007
[714]	valid_0's rmse: 1.04986
[715]	valid_0's rmse: 1.04966
[716]	valid_0's rmse: 1.04949
[717]	valid_0's rmse: 1.04928
[718]	valid_0's rmse: 1.04909
[719]	valid_0's rmse: 1.04888
[720]	valid_0's rmse: 1.04868
[721]	valid_0's rmse: 1.04848
[722]	valid_0's rmse: 1.04827
[723]	valid_0's rmse: 1.04807
[724]	valid_0's rmse: 1.04787
[725]	valid_0's rmse: 1.04767
[726]	valid_0's rmse: 1.04747
[727]	valid_0's rmse: 1.04726
[728]	valid_0's rmse: 1.04706
[729]	valid_0's rmse: 1.04687
[730]	valid_0's rmse: 1.04667
[731]	valid_0's rmse: 1.04646
[732]	valid_0's rmse: 1.04626
[733]	valid_0's rmse: 1.04606
[734]	valid_0's rmse: 1.04587
[735]	valid_0's rmse: 1.04567
[736]	valid_0's rmse: 1.04547
[737]	valid_0's rmse: 1.04528
[738]	vali

[998]	valid_0's rmse: 1.00456
[999]	valid_0's rmse: 1.00443
[1000]	valid_0's rmse: 1.00429
[1001]	valid_0's rmse: 1.00416
[1002]	valid_0's rmse: 1.00405
[1003]	valid_0's rmse: 1.00392
[1004]	valid_0's rmse: 1.0038
[1005]	valid_0's rmse: 1.0037
[1006]	valid_0's rmse: 1.00357
[1007]	valid_0's rmse: 1.00344
[1008]	valid_0's rmse: 1.00332
[1009]	valid_0's rmse: 1.00318
[1010]	valid_0's rmse: 1.00308
[1011]	valid_0's rmse: 1.00297
[1012]	valid_0's rmse: 1.00283
[1013]	valid_0's rmse: 1.00271
[1014]	valid_0's rmse: 1.00259
[1015]	valid_0's rmse: 1.00245
[1016]	valid_0's rmse: 1.00233
[1017]	valid_0's rmse: 1.00222
[1018]	valid_0's rmse: 1.00209
[1019]	valid_0's rmse: 1.00199
[1020]	valid_0's rmse: 1.00188
[1021]	valid_0's rmse: 1.00175
[1022]	valid_0's rmse: 1.00162
[1023]	valid_0's rmse: 1.00151
[1024]	valid_0's rmse: 1.00138
[1025]	valid_0's rmse: 1.00125
[1026]	valid_0's rmse: 1.00115
[1027]	valid_0's rmse: 1.00104
[1028]	valid_0's rmse: 1.00092
[1029]	valid_0's rmse: 1.00079
[1030]	valid

[1304]	valid_0's rmse: 0.974842
[1305]	valid_0's rmse: 0.974765
[1306]	valid_0's rmse: 0.974695
[1307]	valid_0's rmse: 0.974616
[1308]	valid_0's rmse: 0.974538
[1309]	valid_0's rmse: 0.974476
[1310]	valid_0's rmse: 0.974409
[1311]	valid_0's rmse: 0.974331
[1312]	valid_0's rmse: 0.974258
[1313]	valid_0's rmse: 0.974193
[1314]	valid_0's rmse: 0.974113
[1315]	valid_0's rmse: 0.974045
[1316]	valid_0's rmse: 0.973983
[1317]	valid_0's rmse: 0.973906
[1318]	valid_0's rmse: 0.97383
[1319]	valid_0's rmse: 0.973765
[1320]	valid_0's rmse: 0.973687
[1321]	valid_0's rmse: 0.973621
[1322]	valid_0's rmse: 0.973545
[1323]	valid_0's rmse: 0.973479
[1324]	valid_0's rmse: 0.973403
[1325]	valid_0's rmse: 0.973342
[1326]	valid_0's rmse: 0.973278
[1327]	valid_0's rmse: 0.973203
[1328]	valid_0's rmse: 0.973138
[1329]	valid_0's rmse: 0.973062
[1330]	valid_0's rmse: 0.972998
[1331]	valid_0's rmse: 0.972922
[1332]	valid_0's rmse: 0.972863
[1333]	valid_0's rmse: 0.972789
[1334]	valid_0's rmse: 0.972726
[1335]	va

[1612]	valid_0's rmse: 0.959658
[1613]	valid_0's rmse: 0.959622
[1614]	valid_0's rmse: 0.959593
[1615]	valid_0's rmse: 0.959542
[1616]	valid_0's rmse: 0.959504
[1617]	valid_0's rmse: 0.959455
[1618]	valid_0's rmse: 0.959417
[1619]	valid_0's rmse: 0.959379
[1620]	valid_0's rmse: 0.959348
[1621]	valid_0's rmse: 0.959315
[1622]	valid_0's rmse: 0.959278
[1623]	valid_0's rmse: 0.959229
[1624]	valid_0's rmse: 0.959192
[1625]	valid_0's rmse: 0.959161
[1626]	valid_0's rmse: 0.959121
[1627]	valid_0's rmse: 0.959087
[1628]	valid_0's rmse: 0.959037
[1629]	valid_0's rmse: 0.959
[1630]	valid_0's rmse: 0.958963
[1631]	valid_0's rmse: 0.958916
[1632]	valid_0's rmse: 0.958875
[1633]	valid_0's rmse: 0.958842
[1634]	valid_0's rmse: 0.958811
[1635]	valid_0's rmse: 0.958775
[1636]	valid_0's rmse: 0.958728
[1637]	valid_0's rmse: 0.958689
[1638]	valid_0's rmse: 0.958652
[1639]	valid_0's rmse: 0.958613
[1640]	valid_0's rmse: 0.958582
[1641]	valid_0's rmse: 0.958549
[1642]	valid_0's rmse: 0.9585
[1643]	valid_

[1932]	valid_0's rmse: 0.951343
[1933]	valid_0's rmse: 0.951333
[1934]	valid_0's rmse: 0.951316
[1935]	valid_0's rmse: 0.951302
[1936]	valid_0's rmse: 0.951286
[1937]	valid_0's rmse: 0.951273
[1938]	valid_0's rmse: 0.951257
[1939]	valid_0's rmse: 0.951239
[1940]	valid_0's rmse: 0.951225
[1941]	valid_0's rmse: 0.951209
[1942]	valid_0's rmse: 0.951193
[1943]	valid_0's rmse: 0.951181
[1944]	valid_0's rmse: 0.951162
[1945]	valid_0's rmse: 0.95115
[1946]	valid_0's rmse: 0.951136
[1947]	valid_0's rmse: 0.951125
[1948]	valid_0's rmse: 0.951117
[1949]	valid_0's rmse: 0.951104
[1950]	valid_0's rmse: 0.95109
[1951]	valid_0's rmse: 0.951071
[1952]	valid_0's rmse: 0.951058
[1953]	valid_0's rmse: 0.951047
[1954]	valid_0's rmse: 0.951037
[1955]	valid_0's rmse: 0.951024
[1956]	valid_0's rmse: 0.951016
[1957]	valid_0's rmse: 0.950997
[1958]	valid_0's rmse: 0.950984
[1959]	valid_0's rmse: 0.950974
[1960]	valid_0's rmse: 0.950961
[1961]	valid_0's rmse: 0.950947
[1962]	valid_0's rmse: 0.950936
[1963]	val

[2259]	valid_0's rmse: 0.946384
[2260]	valid_0's rmse: 0.946378
[2261]	valid_0's rmse: 0.946356
[2262]	valid_0's rmse: 0.946332
[2263]	valid_0's rmse: 0.946312
[2264]	valid_0's rmse: 0.946287
[2265]	valid_0's rmse: 0.946265
[2266]	valid_0's rmse: 0.946252
[2267]	valid_0's rmse: 0.946231
[2268]	valid_0's rmse: 0.946211
[2269]	valid_0's rmse: 0.946185
[2270]	valid_0's rmse: 0.946161
[2271]	valid_0's rmse: 0.946155
[2272]	valid_0's rmse: 0.946134
[2273]	valid_0's rmse: 0.946111
[2274]	valid_0's rmse: 0.946085
[2275]	valid_0's rmse: 0.946066
[2276]	valid_0's rmse: 0.946041
[2277]	valid_0's rmse: 0.946017
[2278]	valid_0's rmse: 0.946012
[2279]	valid_0's rmse: 0.945991
[2280]	valid_0's rmse: 0.945966
[2281]	valid_0's rmse: 0.945942
[2282]	valid_0's rmse: 0.945922
[2283]	valid_0's rmse: 0.945899
[2284]	valid_0's rmse: 0.945873
[2285]	valid_0's rmse: 0.945868
[2286]	valid_0's rmse: 0.945845
[2287]	valid_0's rmse: 0.94582
[2288]	valid_0's rmse: 0.945801
[2289]	valid_0's rmse: 0.945794
[2290]	va

[2603]	valid_0's rmse: 0.942841
[2604]	valid_0's rmse: 0.942842
[2605]	valid_0's rmse: 0.942832
[2606]	valid_0's rmse: 0.942834
[2607]	valid_0's rmse: 0.942826
[2608]	valid_0's rmse: 0.94283
[2609]	valid_0's rmse: 0.94282
[2610]	valid_0's rmse: 0.942823
[2611]	valid_0's rmse: 0.942813
[2612]	valid_0's rmse: 0.942812
[2613]	valid_0's rmse: 0.94281
[2614]	valid_0's rmse: 0.942812
[2615]	valid_0's rmse: 0.942799
[2616]	valid_0's rmse: 0.942802
[2617]	valid_0's rmse: 0.94279
[2618]	valid_0's rmse: 0.942788
[2619]	valid_0's rmse: 0.94279
[2620]	valid_0's rmse: 0.942775
[2621]	valid_0's rmse: 0.942762
[2622]	valid_0's rmse: 0.942762
[2623]	valid_0's rmse: 0.94276
[2624]	valid_0's rmse: 0.942763
[2625]	valid_0's rmse: 0.942747
[2626]	valid_0's rmse: 0.942745
[2627]	valid_0's rmse: 0.942731
[2628]	valid_0's rmse: 0.942733
[2629]	valid_0's rmse: 0.942721
[2630]	valid_0's rmse: 0.94272
[2631]	valid_0's rmse: 0.942718
[2632]	valid_0's rmse: 0.94272
[2633]	valid_0's rmse: 0.942705
[2634]	valid_0's

[2968]	valid_0's rmse: 0.941355
[2969]	valid_0's rmse: 0.941351
[2970]	valid_0's rmse: 0.941345
[2971]	valid_0's rmse: 0.941342
[2972]	valid_0's rmse: 0.941338
[2973]	valid_0's rmse: 0.941341
[2974]	valid_0's rmse: 0.941336
[2975]	valid_0's rmse: 0.941332
[2976]	valid_0's rmse: 0.941326
[2977]	valid_0's rmse: 0.941322
[2978]	valid_0's rmse: 0.941316
[2979]	valid_0's rmse: 0.941312
[2980]	valid_0's rmse: 0.941309
[2981]	valid_0's rmse: 0.941305
[2982]	valid_0's rmse: 0.9413
[2983]	valid_0's rmse: 0.941295
[2984]	valid_0's rmse: 0.941291
[2985]	valid_0's rmse: 0.941286
[2986]	valid_0's rmse: 0.941282
[2987]	valid_0's rmse: 0.941279
[2988]	valid_0's rmse: 0.941274
[2989]	valid_0's rmse: 0.94127
[2990]	valid_0's rmse: 0.941267
[2991]	valid_0's rmse: 0.941263
[2992]	valid_0's rmse: 0.941257
[2993]	valid_0's rmse: 0.941251
[2994]	valid_0's rmse: 0.941245
[2995]	valid_0's rmse: 0.941238
[2996]	valid_0's rmse: 0.94123
[2997]	valid_0's rmse: 0.941224
[2998]	valid_0's rmse: 0.941218
[2999]	valid

In [75]:
from sklearn.metrics import mean_squared_log_error
from math import sqrt
X_t.head()
score = mean_squared_log_error(Y_test, gbm.predict(X_test))
#score2 = mean_absolute_error(y_test, xgb_val_predictions)
print(sqrt(score))

0.17706161513862864


In [76]:

y_predictions = gbm.predict(X_t)
y_predictions = (10 ** y_predictions)
print("\n",y_predictions)
print(y_predictions.shape)


 [ 1718259.32177992   483094.48951877  3322473.99500019 ...
 25699348.32369843  9709379.07148036  2761981.62639892]
(4398,)


In [78]:
#FOR LGBM
submissions= pd.DataFrame({'id':test.id,'revenue':y_predictions},
                         columns = ['id','revenue'])

submissions.to_csv('submission.csv',index=False)
